In [3]:
import pandas as pd
import numpy as np

import requests

from bs4 import BeautifulSoup

from selenium import webdriver
import re

from pdfquery import PDFQuery

import warnings; warnings.filterwarnings('ignore')

In [6]:
df = pd.read_excel(r"GregMat_list_with_MEANING.xlsx","Sheet1")
renamer = [[0,1],[9,10],[17,18]]
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,Group 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Word,Meaning,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,abound,exist in large numbers or amounts.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,amorphous,without a clearly defined shape or form.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,belie,1. (of an appearance) fail to give a true impr...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624,suspect,1. have an idea or impression of the existence...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
625,NaN,2. doubt the genuineness or truth of.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
626,ploy,a cunning plan or action designed to turn a si...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
627,captious,tending to find fault or raise petty objections.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_words = pd.DataFrame()
for i in range(len(renamer)):
    print(renamer[i])
    df_temp = df.iloc[:,renamer[i]].dropna(how='all')
    df_temp.columns = ['WORD', 'MEANING']
    df_words = df_words.append(df_temp)

[0, 1]
[9, 10]
[17, 18]


In [8]:
df_words['GROUP'] = df_words['WORD'].apply(lambda x: x if 'Group ' in str(x) else np.nan)
df_words[['WORD','GROUP']] = df_words[['WORD','GROUP']].fillna(method = 'ffill')
df_words = df_words[df_words['WORD'].apply(lambda x: x.split()[0] not in ['Group','Take','Word'])]
df_words2 = df_words.dropna(how = 'any').groupby(['GROUP','WORD'])['MEANING'].apply(lambda x: "\n".join(x)).reset_index()
df_words2['GROUP NO'] = df_words2['GROUP'].apply(lambda x: int(x.split()[-1]))
df_words2['GROUP'].value_counts()

Group 22    30
Group 10    30
Group 8     30
Group 7     30
Group 6     30
Group 5     30
Group 4     30
Group 3     30
Group 27    30
Group 26    30
Group 25    30
Group 24    30
Group 23    30
Group 9     30
Group 21    30
Group 2     30
Group 19    30
Group 18    30
Group 17    30
Group 16    30
Group 14    30
Group 13    30
Group 12    30
Group 11    30
Group 20    29
Group 1     29
Name: GROUP, dtype: int64

In [13]:
df_words2

,GROUP,WORD,MEANING,GROUP NO
0,Group 1,abound,exist in large numbers or amounts.,1
1,Group 1,amorphous,without a clearly defined shape or form.,1
2,Group 1,belie,1. (of an appearance) fail to give a true impr...,1
3,Group 1,capricious,given to sudden and unaccountable changes of m...,1
4,Group 1,cerebral,1.of the cerebrum of the brain. \n2. intellect...,1
...,...,...,...,...
773,Group 9,quirky,having or characterized by peculiar or unexpec...,9
774,Group 9,repudiate,1. refuse to accept; reject.\n2. deny the trut...,9
775,Group 9,tact,skill and sensitivity in dealing with others o...,9
776,Group 9,trifling,unimportant or trivial.,9


In [ ]:
def pdf_words(pdf_file):
    pdf = PDFQuery(pdf_file)
    pdf.load()

    # Use CSS-like selectors to locate the elements
    text_elements = pdf.pq('LTTextLineHorizontal')

    # Extract the text from the elements
    text = [t.text for t in text_elements]
    set_no = text[0]
    print(set_no)
    lines = []
    for line in text:
        if len(line.strip())==0: continue
        if 'http' in line.lower(): continue
        if line.split()[0]=='Set': continue

        if len(re.sub("[0-9.]","", line.split()[0]).strip())==0:
            lines.append(line)
        else:
            lines[len(lines)-1] = lines[len(lines)-1] + line

#     for line in lines:
#         print(line)
    
    lines_pdf = pd.DataFrame(lines, columns = ['WORD:Mean'])
    lines_pdf['WORD'] = lines_pdf['WORD:Mean'].apply(lambda x: re.sub("[^a-z]","",str(x).split()[1].lower()))
    lines_pdf['MEANING'] = lines_pdf.apply(lambda x: str(x['WORD:Mean']).split(x['WORD'])[-1][2:].strip(), axis=1)
    
    lines_pdf['GROUP'] = 'Group ' + set_no.split()[-1]
    return lines_pdf[['GROUP','WORD','MEANING']]
pdf_file = '900.pdf'
pdf_df = pdf_words(pdf_file)
pdf_df

In [282]:
driver = webdriver.Chrome(r"C:/Users/70049282/VIA/GRE_Vocab_Scraper/chromedriver.exe")
url = r"https://quizlet.com/in/608598734/all-900-words-gregmat-group-1-to-30-flash-cards/"
driver.get(url)

for button in driver.find_elements_by_tag_name("button"):
    button_text = button.text
    if button.text.strip().title() == 'See More':
        print(button.text)
        button.click()
        break

text = driver.find_element_by_tag_name('body').text
driver.close()

See more


In [283]:
text.split("\n")

['Subjects',
 'Expert solutions',
 'Log in',
 'Sign up',
 '(All 900 Words) GregMat Group 1 to 30',
 '4.9 (20 reviews)',
 'Flashcards',
 'Learn',
 'Test',
 'Match',
 'abound',
 'Click the card to flip 👆',
 'exist in large numbers or amounts.',
 'Click the card to flip 👆',
 '1 / 900',
 'Created by',
 'Kunal_Chand_07',
 'Teacher',
 'Share',
 'GregMAT Vocab Quiz (All 900 Words) :',
 'https://drive.google.com/file/d/1Ng3tB_OMU0Sczy_X6p2xA09E2fC6ipLg/view?usp=sharing',
 '',
 'Original GregMAT Vocab Google Sheet (With Quiz Links): ',
 'https://docs.google.com/spreadsheets/d/1jRATLVV34vATsL4Y67fZZXQc7qZPYc0c0Yk7Bykh4fw/edit#gid=0',
 'Terms in this set (900)',
 'abound',
 'exist in large numbers or amounts.',
 'amorphous',
 'without a clearly defined shape or form.',
 'austere',
 '1. Strict in manner',
 '2. Lack of comforts or luxuries',
 'belie',
 '1. (of an appearance) fail to give a true impression of (something).',
 '2. fail to fulfill or justify (a claim or expectation).',
 'capricious',
 

In [288]:
sentences = text.split("\n")

lines = sentences[sentences.index('Terms in this set (900)')+1:sentences.index('Flickr Creative Commons Images')]

lines[:20]

['abound',
 'exist in large numbers or amounts.',
 'amorphous',
 'without a clearly defined shape or form.',
 'austere',
 '1. Strict in manner',
 '2. Lack of comforts or luxuries',
 'belie',
 '1. (of an appearance) fail to give a true impression of (something).',
 '2. fail to fulfill or justify (a claim or expectation).',
 'capricious',
 'given to sudden and unaccountable changes of mood or behavior.',
 'cerebral',
 '1. of the cerebrum of the brain.',
 '2. intellectual rather than emotional or physical.',
 'congenial',
 "1. (of a person) pleasing or liked on account of having qualities or interests that are similar to one's own.",
 "2. (of a thing) pleasant or agreeable because suited to one's taste or inclination.",
 'conspicuous',
 '1. clearly visible.']

In [299]:
word_meaning_dict = {lines[0]:''}
lines2 = ''
word = lines[0]
multi_count=0
for line in lines[1:]:    
    if line.strip()=='':        
        continue

    if ' ' not in line.strip():
        lines2 = lines2 + "|||" + word + ":::"
        if word_meaning_dict[word]=='':
            word_meaning_dict[word]+=line.strip()
        else:    
            word = line
            word_meaning_dict[word] = ''
            
        
        
    else:
        word_meaning_dict[word] += line.strip()
        lines2 = lines2 + line + "\n"
        

    
word_df = pd.DataFrame(word_meaning_dict.items(), columns = ['WORD','MEANING'])
# word_meaning_dict['belie']

In [300]:
word_df

,WORD,MEANING
0,abound,exist in large numbers or amounts.
1,amorphous,without a clearly defined shape or form.
2,austere,1. Strict in manner2. Lack of comforts or luxu...
3,belie,1. (of an appearance) fail to give a true impr...
4,capricious,given to sudden and unaccountable changes of m...
...,...,...
894,cogitate,to think deeply
895,nominal,"(adj.) existing in name only, not real; too sm..."
896,plucky,courageous; spunky
897,grovel,"(v) to humble oneself, act in a fearful and se..."


In [301]:
word_df.to_excel(r"Test.xlsx", index = False)
word_df.iloc[25:40,:]


,WORD,MEANING
25,puerile,childishly silly and immature.
26,quixotic,extremely idealistic; unrealistic and impracti...
27,spendthrift,"a person who spends money in an extravagant, i..."
28,taciturn,(of a person) reserved or uncommunicative in s...
29,wary,feeling or showing caution about possible dang...
30,adulterate,render (something) poorer in quality by adding...
31,advocate,a person who publicly supports or recommends a...
32,aggrandize,"increase the power, status, or wealth of."
33,alacrity,cheerful willingness and liveliness
34,ambivalent,having mixed feelings or contradictory ideas a...


In [173]:

# driver = webdriver.Chrome("C:/Users/Admin/Desktop/GREGMAT PREP/chromedriver.exe")
# driver.get(url)
# text = driver.find_element_by_tag_name('body').text

def text_miner(word):
    print(word)
    print(word_meaning_dict[word])
    print("\n")
    url = f'https://www.google.com/search?q={word}%20word%20meaning#dobc=en'

    response = requests.get(url, verify = False)
    a = response.text
    soup = BeautifulSoup(a, 'html.parser')
    text = soup.get_text()
    return text

def synonym_mining(word):
    print(word)
    print(word_meaning_dict[word])
    
    text2 = text.split("People also ask")[0]
    text3 = text2.split("synonyms: ")[-1]
    
    if 'synonyms' in text:
        return text3
    else:
        return np.nan
    

In [174]:
    
word_df['GOOGLE_TEXT'] = word_df['WORD'].apply(text_miner)
word_df

abound
 exist in large numbers or amounts.


amorphous
 without a clearly defined shape or form.


austere
 1. Strict in manner 2. Lack of comforts or luxuries


belie



capricious
 given to sudden and unaccountable changes of mood or behavior.


cerebral
 1. of the cerebrum of the brain. 2. intellectual rather than emotional or physical.


congenial
 1. (of a person) pleasing or liked on account of having qualities or interests that are similar to one's own. 2. (of a thing) pleasant or agreeable because suited to one's taste or inclination.


conspicuous
 1. clearly visible. 2. attracting notice or attention.


cursory
 hasty and therefore not thorough or detailed.


daunting
 seeming difficult to deal with in prospect; intimidating.


deify
 worship or regard as a god.


didactic
 in the manner of a teacher, particularly so as to appear patronizing.


disseminate
 spread (something, especially information) widely.


feasible
 possible to do easily or conveniently.


flout
 openly di

fanciful



frivolous



heterogeneous
 diverse in character or content.


imperious



impertinent
 1. not showing proper respect; rude. 2. not pertinent to a particular matter; irrelevant.


invasive
 tending to spread very quickly and undesirably or harmfully.


irresolute



laudable
 (of an action, idea, or aim) deserving praise and commendation.


lax
 1. not sufficiently strict, severe, or careful. 2. (of the limbs or muscles) relaxed.


marginalize



panache
 flamboyant confidence of style or manner.


plodding
 slow-moving and unexciting.


prosaic
 having or using the style or diction of prose as opposed to poetry; lacking imaginativeness or originality.


remedial
 giving or intended as a remedy or cure.


restive



sporadic



stigmatize
 describe or regard as worthy of disgrace or great disapproval.


undermine



utterly
 completely and without qualification; absolutely.


weary
 feeling or showing extreme tiredness, especially as a result of excessive exertion.


zealo

quirky
 having or characterized by peculiar or unexpected traits or aspects.




KeyboardInterrupt: 

In [165]:
word = 'construe'
url = f'https://www.google.com/search?q={word}%20word%20meaning#dobc=en'

response = requests.get(url, verify = False)
a = response.text
soup = BeautifulSoup(a, 'html.parser')
text = soup.get_text()
text2 = text.split("People also ask")[0]
text3 = text2.split("synonyms: ")[-1]
text4 = text3.split(".")[-1]
print(text3)
print("\n\n\n")

interpret, understand, read, see, take, take to mean, parse, render, analyse, explain, elucidate, gloss, decodeanalyse the syntax of (a text, sentence, or word).'both verbs can be construed with either infinitive'






In [167]:
text4

"'both verbs can be construed with either infinitive'"